In [1]:
import json
import jsonlines
import sys
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


合并多个文件的先验知识

创建数据（带有先验知识的Prompt）以微调LLM

测试模型的先验知识生成效果：一次提问

In [2]:
import os
import sys
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type, TemplateType
)

from custom import CustomModelType, CustomTemplateType

# model_dir = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1/lr=1.1e-4/lora_rank=8/split_type=8:2-train_ratio=1.0-20240116-00:52:51/checkpoint-609'
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
template_type = CustomTemplateType.mistral # get_default_template_type(model_type)
print(f"Template type: {template_type}")

model, tokenizer = get_model_tokenizer(
    model_type, torch_dtype=torch.float16, model_kwargs={'device_map': 'auto'})
model.generation_config.max_new_tokens = 2048 # 512
template = get_template(template_type, tokenizer)


2024-04-06 12:44:30,647 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-06 12:44:30,649 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-06 12:44:30,675 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "num

Template type: mistral


/home/hanlv/miniconda3/envs/swift/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


KeyboardInterrupt: 

In [3]:
from tqdm.notebook import tqdm
data_search_llm = prompt_rag.load_search_llm_tmp()
K=5

for i in tqdm(range(len(data_search_llm))):
    item = data_search_llm[i]

    if item.get(f"prior_knowledge_mixtral2") is None:
        continue
    p1 = item["prior_knowledge_mixtral2"]

    if (len(tokenizer(p1)["input_ids"]) < 200):
        claim = data_search[i]["claim"]
        search_results = data_search[i][f"{search_engine}_search_results"]

        model_name = 'mixtral'
        prompt = prompt_rag.get_prompt_for_generating_prior_knowledge(
            claim, data_search[i]["date"], search_engine, search_results, model_name,
            K=K, sort=False, 
            ids=None
        )
        response, history = inference(
            model, template, prompt, 
        )
        item["prior_knowledge_mixtral3"] = response.strip()

prompt_rag.save_search_llm_tmp(data_search_llm)


  0%|          | 0/14384 [00:00<?, ?it/s]